In [3]:
from torch.backends.cuda import allow_fp16_bf16_reduction_math_sdp

from core.PC_NET import PCNet
import torch
from torch.utils.data import DataLoader
from core.dataset import collate_fn
import yaml
from sklearn.metrics import f1_score, classification_report
from core.config import punct_label2id

model = PCNet.load_from_checkpoint('../logs_all/logs/PCNet_lr1e-4_layers0/last-checkpoint.ckpt')
model.eval()

dataset = torch.load('../data/pt_datasets/test.pt')
dataloader = DataLoader(dataset, batch_size=512, shuffle=False, collate_fn=collate_fn)

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cap_preds = []
cap_labels_all = []

punct_labels_all = []
punct_preds_ = []


for batch in dataloader:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    punct_labels = batch["punct_labels"].to(device)
    cap_labels = batch["cap_labels"].to(device)
    subword_tokens = batch["subword_tokens"]

    punct_logits, cap_logits = model(input_ids, attention_mask)
    cap_labels_pred = torch.argmax(cap_logits, dim=-1)
    punct_labels_pred = torch.argmax(punct_logits, dim=-1)
    valid_mask = (attention_mask.view(-1) == 1) & (cap_labels.view(-1) != -100)
    cap_preds.extend(cap_labels_pred.view(-1)[valid_mask].tolist())
    cap_labels_all.extend(cap_labels.view(-1)[valid_mask].tolist())

    punct_labels_all.extend(punct_labels.view(-1)[valid_mask].tolist())
    punct_preds_.extend(punct_labels_pred.view(-1)[valid_mask].tolist())



